In [58]:
import pandas as pd
import nltk
import re
import numpy as np
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
#Json file option
#filejson = "C:/Users/danie/Desktop/bbc_news_list_uk.json"
filecsv = r"C:\Users\danie\Documents\GitHub\Masters-Thesis\bbc_news_list_uk.csv"
#filecsv = r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/bbc_news_list_uk.csv"
article_df = pd.read_csv(filecsv)
article_df = article_df.assign(Article_Number=range(len(article_df)))
article_df = article_df.reset_index()
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15825 entries, 0 to 15824
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              15825 non-null  int64 
 1   tags               1 non-null      object
 2   title              15825 non-null  object
 3   news_post_date     15825 non-null  object
 4   raw_content        15468 non-null  object
 5   content            15468 non-null  object
 6   url                15825 non-null  object
 7   author             1453 non-null   object
 8   language           15825 non-null  object
 9   id                 15825 non-null  object
 10  region             15488 non-null  object
 11  short_description  15825 non-null  object
 12  category           15825 non-null  object
 13  crawled_at         15825 non-null  object
 14  Article_Number     15825 non-null  int32 
dtypes: int32(1), int64(1), object(13)
memory usage: 1.8+ MB


In [60]:
#this code assumes the first four digits are the year. can be changed for last of middle
year = article_df['news_post_date'].str[:4]
article_df['year']=year

In [61]:
#filename = 'articles.pkl'

#article_df = pd.read_pickle(filename)
#article_df = article_df.assign(Article_Number=range(len(article_df)))
#article_df = article_df.reset_index()
#article_df.info()

In [62]:
# tokenize sentences in an article 
import re

def split_sentences(article, article_id, year):
    pattern = r'(?<=[a-z0-9"]) *[.?!] *(?=[A-Z])'
    article = re.sub(pattern, r'\g<0> ', article)
    sentences = nltk.sent_tokenize(article)
    sentences_with_id = [(sentence, article_id, year) for sentence in sentences]
    return sentences_with_id

sentences_list = []

# add sentences to a new DF along with article ID 
for article, article_id, year in article_df[['content','Article_Number', 'year']].values:
    sentences = split_sentences(str(article), article_id, year)
    sentences_list.extend(sentences)

sentences_df = pd.DataFrame(sentences_list, columns= ['sentences', 'article_id', 'year'])

In [63]:
def pronoun_occurances(text):
    """ This function will count the number of female and male pronoun occurences in a given sentence."""
    pattern_m = r'(\s|^)(["\']?)(he|his|him|he\'s|hes)\2\b' #this regex will capture he/his as standalone words within a string but also at beginning of sentence
    matches_m = re.findall(pattern_m, text, re.IGNORECASE) #IGNORECASE is necessary to make sure that it picks up the pronouns at the beginning of a sentence
    pattern_f = r'(\s|^)(["\']?)(she|her|hers|shes|she\'s)\2\b'
    matches_f = re.findall(pattern_f, text, re.IGNORECASE)
    #pattern_n = r'(\s|^|")(["\']?)(they|them|their|theirs|their\'s)\2\b' #
    #matches_n = re.findall(pattern_n, text, re.IGNORECASE)
    count_m = len(matches_m)
    count_f = len(matches_f)
    #count_n = len(matches_n)
    #count_u = count_n + count_f + count_m
    return count_f, count_m #, count_n, count_u

reporting_words = ["acknowledge", " add", " address", " admit", " announce", " argue", " believe", " claim", " conclude", " confirm", " continue", " declare", " describe", " ensure", " estimate", " explain", " find", " indicate", " inform", " insist", " note", " point", " predict", " provide", " release", " reply", " report", " respond", " say", " state", " suggest", " tell", " testify", " think", " tweet", " warn", " write"]

 If sent contains a reporting word then
    look to the word before for a name or a pronoun
        If found and a name find gender 
            once gender is found clasify the sentence
        if found a pronoun then classify sentence
        else   
            look after the word for the same and follow the same pattern
                else follow the rest of the pattern
Count the number of male, female, and neutral pronouns

*****Look to see if the sentence contains a proper noun and add the gender to the ocunt
    I think this should be added to the preprocessing because we are already looking up the POS tags so it will use less bullshit

In [64]:
#create a variable applying the function of pronoun occurence
sent = sentences_df['sentences'].apply(pronoun_occurances)
# Create two new columns in sentences DF from the tuple output in "sent"
sentences_df['female_count'] = [x[0] for x in sent]
sentences_df['male_count']= [x[1] for x in sent]
#sentences_df['neutral_count']= [x[2] for x in sent]
#sentences_df['u_count']= [x[3] for x in sent]

#Bug is fixed and now it counts properly
sentences_df

,sentences,article_id,year,female_count,male_count
0,The heroin substitute methadone can be used as...,0,2010,0,0
1,Earlier this year a debate broke out in Scotla...,0,2010,0,0
2,"But a group of 40 specialists, including unive...",0,2010,0,0
3,So what do recovering addicts think?,0,2010,0,0
4,Chris used methadone for five years to help we...,0,2010,0,3
...,...,...,...,...,...
225660,Similar moves in Europe have sparked cries of ...,15824,2010,0,0
225661,The lead in the Daily Mail is a claim that cou...,15824,2010,0,0
225662,It says authorities are using the information ...,15824,2010,0,0
225663,The Daily Express leads with a warning from US...,15824,2010,0,0


In [65]:
def compare_count(male_col, female_col): 
    """This function compares the count of female to male pronouns. It will output "1" if male count bigger
    than female count, "neutral" if the count is equal, and "female" if there is a higher female count. 
    The function returns strings because we need categorical variables for log reg to run"""
    if female_col > male_col:
        return "0"
    elif male_col > female_col:
        return "1"
    else: 
        return None
sentences_df['col_type'] = sentences_df.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)
sentences_df

,sentences,article_id,year,female_count,male_count,col_type
0,The heroin substitute methadone can be used as...,0,2010,0,0,None
1,Earlier this year a debate broke out in Scotla...,0,2010,0,0,None
2,"But a group of 40 specialists, including unive...",0,2010,0,0,None
3,So what do recovering addicts think?,0,2010,0,0,None
4,Chris used methadone for five years to help we...,0,2010,0,3,1
...,...,...,...,...,...,...
225660,Similar moves in Europe have sparked cries of ...,15824,2010,0,0,None
225661,The lead in the Daily Mail is a claim that cou...,15824,2010,0,0,None
225662,It says authorities are using the information ...,15824,2010,0,0,None
225663,The Daily Express leads with a warning from US...,15824,2010,0,0,None


In [66]:
#remove columns with "None" in the col_type 
sentences_df = sentences_df[sentences_df["col_type"].notnull()]
#sentences_df

In [67]:
sentences_df.sort_values("female_count", ascending = False)

,sentences,article_id,year,female_count,male_count,col_type
175442,Mrs Jones previously admitted failing to notif...,12116,2010,7,1,0
60832,She also said that she had cut her mother's na...,4004,2010,7,0,0
23016,"In her appeal, she claimed she only pleaded gu...",1339,2010,7,1,0
46585,"""She left her job, she left someone in her fla...",2967,2010,7,1,0
17800,I even made her do one in her bikini - which s...,971,2010,6,0,0
...,...,...,...,...,...,...
86076,The Telegraph quotes PR experts who say Green ...,5777,2010,0,2,1
86090,Many economists also believe that the previous...,5778,2010,0,1,1
86143,He is in a state of deep shock after his house...,5781,2010,0,2,1
86144,Anwar doesn't know where his wife and children...,5781,2010,0,2,1


0 - female 
1 - male

**Classifier Preparation**

In [122]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

name_probability_list = {}
#This needs to be pickled

In [160]:
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from genderize import Genderize
from tqdm.notebook import tqdm
import time

#Sentence Encoding
def tidy_text(sentence, remove_stopwords = True):
    #!need to change all to nethods and time them
    print(sentence)
    regex_start = time.perf_counter()
    Male_count = 0
    male_list= {"man", "men", "mister", "he", "him", "Mr.", "he", "his","he's", "hes", "father", "dad", "daddy", "grandpa", "grandfather", "husband"}
    female_list ={"woman", "women", "missus", "misses", "Ms.", "Mrs.", "her", "she", "hers", "mother", "mom", "mommy", "aunt", "grandmother", "grandma", "wife", "wive"}
    Female_count = 0
    APIcallfail= 0

    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub("\d+", " ", sentence)
    
    #changed the number detection code
    sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)
    regex_end = time.perf_counter()
    print(f"Ran regex in {regex_start - regex_end:0.4f} seconds")

    # Tokenize each word
    sentence =  nltk.TweetTokenizer().tokenize(sentence)

    nltk.tag.pos_tag(sentence)
    tagged_sentence = nltk.tag.pos_tag(sentence)
    lemma = nltk.stem.WordNetLemmatizer()

    token_ = time.perf_counter()
    print(f"Ran tokenization and tagging in {regex_end - token_:0.4f} seconds")

    pn_tags = {'NNP', 'NNPS'}
    new_words = []

    for word, tag in tagged_sentence: 
        if tag not in pn_tags: 
            if tag.startswith("V"):
                lemmas = lemma.lemmatize(word, "v")
            else: 
                lemmas = lemma.lemmatize(word)
            new_words.append((lemmas))
        else:
            try: 
                word_gender = name_probability_list.get(word)
                if word_gender is None:
                    word_gender = Genderize().get1(word).get('gender')
                    #What do I do if the genderize is looking for a name that doesn't exist 
                    #Test once i have more API calls
                    name_probability_list[word] = word_gender

                if word_gender == 'male':
                    Male_count += 1
                if word_gender== 'female':
                    Female_count += 1
            except Exception as exception:
                APIcallfail +=1
    sentence = new_words

    lemma = time.perf_counter()
    print(f"Ran lemmas and genderize in {token_ - lemma:0.4f} seconds")

    new_text = []
    for word in sentence:
        contraction = contractions.get(word)
        if contraction is None:
            new_text.append(word)
        else:
            for new_word in contraction.split():
                new_text.append(new_word)
    sentence = new_text

    contraction_time = time.perf_counter()
    print(f"Ran contractions in {lemma - contraction_time:0.4f} seconds")
          
    for w in sentence:
        if w in male_list:
            Male_count += 1
        if w in female_list:
            Female_count += 1

    # remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        sentence = [w for w in sentence if not w in stops]


    new_sent = [x for x in sentence if x not in male_list]
    new_sent = [x for x in new_sent if x not in female_list]

    sentence = new_sent

    #!  here we check for words from the gender words and we ensure they are removed from the tokens. this will ensure we have 0 leakage
    return sentence, Male_count, Female_count, APIcallfail

text = "she she's she'll he he he purple"
tidy_text(text)


she she's she'll he he he purple
Ran regex in -0.0000 seconds
Ran tokenization and tagging in -0.0038 seconds
Ran lemmas and genderize in -0.0002 seconds
Ran contractions in -0.0000 seconds


(['purple'], 3, 3, 0)

In [161]:
sentences_df['encoded_sentences'] = sentences_df['sentences'].apply(tidy_text)

Chris used methadone for five years to help wean him off heroin but felt he had to quit the drug substitute last October"I just thought, I've got to get a grip here, because I've been in and out of prison since I was 16 - that's half my life," he told The Report.
Ran regex in -0.0001 seconds
Ran tokenization and tagging in -0.0083 seconds
Ran lemmas and genderize in -2.0176 seconds
Ran contractions in -0.0008 seconds
"I heard about Ratho Hall, and it's given me the wake-up call that I needed," he said.
Ran regex in -0.0001 seconds
Ran tokenization and tagging in -0.0099 seconds
Ran lemmas and genderize in -0.7995 seconds
Ran contractions in -0.0002 seconds
Chris was prescribed methadone, a synthetic opiate that is taken orally, to wean him off his heroin addiction but although methadone can help addicts stabilise their lives, it is also addictive.
Ran regex in -0.0001 seconds
Ran tokenization and tagging in -0.0119 seconds
Ran lemmas and genderize in -0.3984 seconds
Ran contractions in

KeyboardInterrupt: 

In [26]:
sentences_df

,sentences,article_id,year,female_count,male_count,col_type,encoded_sentences
4,Chris used methadone for five years to help we...,0,2010,0,3,1,"[use, methadone, five, year, help, wean, heroi..."
6,"""I heard about Ratho Hall, and it's given me t...",0,2010,0,1,1,"[I, hear, give, wake, call, I, need, say]"
7,"Chris was prescribed methadone, a synthetic op...",0,2010,0,2,1,"[prescribe, methadone, synthetic, opiate, take..."
12,"In this, the Community Safety Minister, Fergus...",0,2010,0,1,1,"[In, explain, want, see, new, vision, drug, tr..."
14,"Chris, who started to use heroin when he was 1...",0,2010,0,2,1,"[start, use, heroin, talk, addiction, When, he..."
...,...,...,...,...,...,...,...
225598,"""For Diego, it is good for him to see the norm...",15819,2010,0,2,1,"[For, good, see, normal, rearing, process, wit..."
225601,The new monkey will stay at the zoo until he r...,15819,2010,0,3,1,"[The, new, monkey, stay, zoo, reach, sexual, m..."
225605,He died in hospital.,15820,2010,0,1,1,"[He, die, hospital]"
225645,"He said: ""Local authorities need to recognise ...",15823,2010,0,1,1,"[He, say, authority, need, recognise, regional..."


**Split the DF by Year**

In [27]:
def split_df(df, column_name): 
    """Split Sentences DF by the year column so that we can do the LR by year"""

    #get all unique values in col
    col_vals = df[column_name].unique()

    split = [df[df[column_name] == value] for value in col_vals]
    
    #Return the list of split dataframes
    return split

#use the new function to split the original sentences df 
splitted_df = split_df(sentences_df, 'year')

#print each of the DFs
for df_year_data in splitted_df:
    display(df_year_data)

,sentences,article_id,year,female_count,male_count,col_type,encoded_sentences
4,Chris used methadone for five years to help we...,0,2010,0,3,1,"[use, methadone, five, year, help, wean, heroi..."
6,"""I heard about Ratho Hall, and it's given me t...",0,2010,0,1,1,"[I, hear, give, wake, call, I, need, say]"
7,"Chris was prescribed methadone, a synthetic op...",0,2010,0,2,1,"[prescribe, methadone, synthetic, opiate, take..."
12,"In this, the Community Safety Minister, Fergus...",0,2010,0,1,1,"[In, explain, want, see, new, vision, drug, tr..."
14,"Chris, who started to use heroin when he was 1...",0,2010,0,2,1,"[start, use, heroin, talk, addiction, When, he..."
...,...,...,...,...,...,...,...
225598,"""For Diego, it is good for him to see the norm...",15819,2010,0,2,1,"[For, good, see, normal, rearing, process, wit..."
225601,The new monkey will stay at the zoo until he r...,15819,2010,0,3,1,"[The, new, monkey, stay, zoo, reach, sexual, m..."
225605,He died in hospital.,15820,2010,0,1,1,"[He, die, hospital]"
225645,"He said: ""Local authorities need to recognise ...",15823,2010,0,1,1,"[He, say, authority, need, recognise, regional..."


,sentences,article_id,year,female_count,male_count,col_type,encoded_sentences
21436,One woman is quoted as saying the gangsters th...,1230,2012,3,0,0,"[One, woman, quote, say, gangster, threaten, b..."
21438,He has also called for the criminals to be tra...,1230,2012,0,1,1,"[He, also, call, criminal, track, punish]"
85224,"However, Mr Fico said that he would still try ...",5717,2012,0,1,1,"[However, say, would, still, try, form, govern..."
85225,"Referring to his party's 63 seats, he said: ""T...",5717,2012,0,2,1,"[Referring, party, seat, say, This, number, gi..."
85232,"""The citizens of Slovakia have voted for respo...",5717,2012,1,0,0,"[The, citizen, vote, responsibility, add]"
85233,"With partners Freedom and Solidarity, the Chri...",5717,2012,1,0,0,"[With, partner, ethnic, party, party, enough, ..."
85239,In a television address shortly before polls o...,5717,2012,0,1,1,"[In, television, address, shortly, poll, open,..."
85241,"""Any mash-up is better than Fico,"" said SaS le...",5717,2012,0,1,1,"[Any, mash, better, say, leader, say, party, w..."
85989,But President Ivan Gasparovic said he would gi...,5773,2012,0,1,1,"[But, say, would, give, first, chance, form, g..."
85990,Mr Gasparovic said he wanted to respect the tr...,5773,2012,0,1,1,"[say, want, respect, tradition, give, leader, ..."


In [17]:
#pd.get_dummies(sentences_df.year)
#rated_dummies = pd.get_dummies((sentences_df).year)
#sentences_df = pd.concat([sentences_df, rated_dummies], axis=1)

**Binary Logistic Regression**

In [28]:
#vectorizer = TfidfVectorizer(max_features= 1000, lowercase=False, tokenizer=False)
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

#X_train = tfidf.fit_transform(X_train)
#X_test = tfidf.fit_transform(X_test)

In [ ]:
#GridSearchCV Packages - this still needs changing and fitting into the LR function!
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings("ignore")


# parameter grid
parameters = {"penalty": ['l1','l2'], 
              "C": np.logspace(-3,3,7),
              "solver": ['newton-cg', 'lbfgs', 'liblinear'],
}

#GridSearchCV
logreg = LogisticRegression()
clf_logreg = GridSearchCV(logreg, 
                          param_grid = parameters, 
                          scoring = "accuracy", 
                          cv = 10)

clf_logreg.fit(X_train, y_train)

**Logistic Regression Classifier by Year**

In [128]:
def logistic_regression_year(df, target_col, text_col, year_col):
    years = df[year_col].unique()
    results = {}

    #split data 
    for year in years:
        df_year = df.loc[df[year_col]==year].copy()
        df_year['text'] = df_year[text_col].apply(lambda x: ' '.join(map(str, x)))
        X = df_year[text_col].apply(lambda x: str(x))
        y = df_year[target_col]

    #train test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        tfidf = TfidfVectorizer()
        X_train_tfidf = tfidf.fit_transform(X_train)
        X_test_tfidf = tfidf.transform(X_test)

    #run the classifier 
        clf = LogisticRegression()
        clf.fit(X_train_tfidf, y_train)
        y_pred = clf.predict(X_test_tfidf)

    #performance 
        accuracy = clf.score(X_test_tfidf, y_test)
        class_report = classification_report(y_test, y_pred, zero_division = 0)
        results[year] = {'accuracy': accuracy, 'classification_report': class_report}
        print(f"Year: {year}")
        print(f"Accuracy: {accuracy}")
        print(f"Classification Report:\n{class_report}")

        #coefficients
        coefs = clf.coef_[0]
        sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
        high_coef = sorted_coef[:5]
        low_coef = sorted_coef[-5:]
        

        #print coefficient results 
        print(f"\n Highest coefs:")
        for i in high_coef: 
            print(i)
        
        print(f"\n Lowest coefs:")
        for i in low_coef: 
            print(i)
    

In [129]:
logistic_regression_year(sentences_df, 'col_type', 'encoded_sentences', 'year')

Year: 2010
Accuracy: 0.8566366305558368
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.40      0.55      2183
           1       0.85      0.99      0.91      7694

    accuracy                           0.86      9877
   macro avg       0.87      0.69      0.73      9877
weighted avg       0.86      0.86      0.83      9877


 Highest coefs:
('he', 15.891551900218763)
('his', 7.603456046330884)
('man', 4.17082789968222)
('wife', 3.5879416740474666)
('soldier', 2.378104672128594)

 Lowest coefs:
('girl', -5.560419224363694)
('husband', -6.233141993018479)
('woman', -7.343204828079678)
('her', -9.555410225636514)
('she', -18.592092822166705)
Year: 2012
Accuracy: 0.4
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.40      1.00      0.57         2

    accuracy                           0.40         5
   macro avg       0.20

**Binary Logistics Regression (OLD VERSION)**

In [25]:
#finetuned model with best hyperparameters 
logreg1 = LogisticRegression(C=0.1, penalty = "l2", solver = "liblinear", max_iter = 5000)

LogisticRegression(C = 0.1, penalty = "l2", solver = "liblinear", max_iter = 5000)

In [26]:
fitted_logreg = logreg1.fit(X_train, y_train)
fitted_logreg

LogisticRegression(C=0.1, max_iter=5000, solver='liblinear')

In [27]:
prediction = fitted_logreg.predict(X_test)

In [28]:
print("Accuracy:",fitted_logreg.score(X_test, y_test))

Accuracy: 0.8390002023881805


L2 is better to prevent overfitting than L1 because it punishes errors more (squares them as opposed to taking the absolute value). GridSearchCV said either L1 or L2 were equally as good though. 

In [29]:
coefs = fitted_logreg.coef_[0]

#male is 1 and female is 0

sorted_coef = sorted((zip(tfidf.get_feature_names(), coefs)), key = lambda x: x[1], reverse=True)

high_coef = sorted_coef[:10]
low_coef = sorted_coef[-10:]

print("highest coefs")
for i in high_coef: 
    print(i)

print("lowest coefs")
for i in low_coef: 
    print(i)

highest coefs
('He', 7.497867226565254)
('His', 2.791468567597014)
('man', 1.7939417368758859)
('wife', 1.2597679909365127)
('shoot', 0.8793346468852085)
('soldier', 0.8253443935307779)
('security', 0.552288517084893)
('general', 0.5360865508650358)
('minister', 0.5225045885836616)
('boy', 0.5212790475191019)
lowest coefs
('home', -0.8708266336604314)
('actress', -1.0912651232977952)
('child', -1.13564985240973)
('daughter', -1.6608630837522824)
('mother', -2.061830887374049)
('girl', -2.2616077753072013)
('husband', -2.5359911903913552)
('woman', -3.8458811360803873)
('Her', -3.9024438512883517)
('She', -10.14035943687664)


Interestingly, male coefficient for neutral is higher than the female. Otherwise, as expected, the coefficients for female and male each are correspondingly high for each gender. 

In [30]:
print(metrics.accuracy_score(y_test, prediction)) #accuracy 
print(metrics.confusion_matrix(y_test, prediction)) #confusion matrix
print(metrics.classification_report(y_test, prediction))

0.8390002023881805
[[ 559 1569]
 [  22 7732]]
              precision    recall  f1-score   support

           0       0.96      0.26      0.41      2128
           1       0.83      1.00      0.91      7754

    accuracy                           0.84      9882
   macro avg       0.90      0.63      0.66      9882
weighted avg       0.86      0.84      0.80      9882

